### Stage 01 data_ingestion

In [1]:
import os
import sys
%pwd

'd:\\collab_projects\\South_German_Credit_Risk\\notebook'

In [3]:
os.chdir("../")

%pwd

'd:\\collab_projects\\South_German_Credit_Risk'

### Create the Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url: str 
    local_data_file: Path 
    unzip_dir: Path

In [6]:
from South_German_Bank.constants import *
from South_German_Bank.utils.common import read_yaml, create_directories

### Create the ConfigurationManager

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir,
          )

          return data_ingestion_config

### Create the Component

In [8]:
import os
import urllib.request as request
import zipfile
from South_German_Bank.logging import logger
from South_German_Bank.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded successfully with following headers: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

### Create the Pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config\config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/data_ingestion]
[{asctime}s: INFO: 1430679460: artifacts/data_ingestion/data.zip downloaded successfully with following headers: 
Connection: close
Content-Length: 17783
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "46235576a5239af6c792cbd0e20d401fb3e42666a76e92919d6068b5262358f1"
Last-Modified: Sun, 30 Jul 2023 05:44:10 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 16B2:16DBAC:2413E3:326979:64C5F8A8
Accept-Ranges: bytes
Date: S